In [5]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity
    
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    
class Promotion(ABC):
    @abstractmethod
    def discount(self, order):
        """返回折扣金额（正值）"""

class FidelityPromo(Promotion):
    """为积分为1000或以上的顾客提供5%折扣"""
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0
    
class BulkItemPromo(Promotion):
    """单个商品为20个或以上时提供10%折扣"""
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount
    
class LargeOrderPromo(Promotion):
    """订单中的不同商品达到10个或以上时提供7%折扣"""
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [6]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
       LineItem('apple', 10, 1.5),
       LineItem('watermellon', 5, 5.0)]
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [7]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [8]:
banana_cart = [LineItem('banana', 30, .5),
              LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [9]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [10]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [18]:
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

    
def fidelity_promo(order):
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
        return discount
    
def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10 :
        return order.total() * .07
    return 0



In [12]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [13]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [16]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [19]:
Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [21]:
Order(joe, cart)

<Order total: 42.00 due: 42.00>

In [24]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    return max(promo(order) for promo in promos)
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [25]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [26]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [28]:
promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']

In [29]:
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [37]:
import inspect
import promotions
promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]

In [38]:
promos

[<function promotions.bulk_item_promo(order)>,
 <function promotions.fidelity_promo(order)>,
 <function promotions.large_order_promo(order)>]

In [39]:
class MacroCommand:
    def __init__(self, commands):
        self.commands = list(commands)
        
    def __call__(self):
        for command in self.commands:
            command()

In [42]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)

In [43]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

## 总结
- 可以使用函数代替经典的使用类实现的策略模式
- 第七章还使用了装饰器来动态实现策略